<a href="https://colab.research.google.com/github/BALAMANIKANTA29/Fmml__labs__projects__assginments/blob/main/fmml_m1l2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


Yes, averaging the validation accuracy across multiple splits typically leads to more consistent and reliable results. This approach is commonly known as cross-validation. Here's why it helps:

1. Reduces Variance:

When you split your dataset into training and validation sets only once (e.g., a single train-test split), the model’s performance can depend heavily on that specific split. If the split is not representative of the overall dataset (e.g., one split might have more difficult examples than another), the accuracy could be biased.

By averaging the accuracy over multiple splits, such as in k-fold cross-validation, you reduce the effect of any single "lucky" or "unlucky" split, leading to more robust results.


2. Generalization:

Multiple splits ensure that the model sees all the data points for training and validation at some point. This means you’re testing the model's ability to generalize across the entire dataset rather than just one subset.

For example, in k-fold cross-validation, the dataset is split into k equally sized subsets. Each subset serves as the validation set once while the model trains on the other k-1 subsets. This ensures that the final averaged accuracy reflects the model’s performance across the entire dataset.


3. Handles Data Imbalance:

If you have an imbalanced dataset, a single split might not capture the full range of the minority class. Cross-validation helps by ensuring that different parts of the data are used both in training and validation, which can lead to more reliable accuracy estimates.

4. Statistical Stability:

Averaging accuracy across multiple splits provides a more statistically stable estimate of the model’s performance, smoothing out random fluctuations due to overfitting or underfitting on certain subsets.

Example: K-Fold Cross-Validation

In a standard k-fold cross-validation:

1. Split the data into k equal subsets.


2. Train the model k times, each time leaving one subset out as the validation set and training on the remaining k-1 subsets.


3. Calculate the accuracy for each of the k splits.


4. Average the accuracies to get a final performance metric.



This approach often gives a more realistic sense of how the model will perform on unseen data.

Cons:

Time: Cross-validation can be computationally expensive, especially for large datasets or complex models, since the model has to be trained multiple times.

Overfitting: If not done carefully (e.g., using the same data for feature selection before cross-validation), it could still lead to some overfitting.


Would you like to see an example using cross-validation on the MNIST dataset or any other specific dataset? I can walk you through the steps or run a demonstration!

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Yes, averaging validation accuracy across multiple splits, like in cross-validation, generally provides a more accurate and reliable estimate of test accuracy for several reasons:

1. Better Estimate of Model Generalization:

Cross-validation (e.g., k-fold cross-validation) evaluates the model on different portions of the data, ensuring that the model's performance is not biased by a single train-test split. Since every data point gets used in both training and validation at some point, the result is a more reliable estimate of how the model will perform on unseen (test) data.

This process mimics the model's behavior on new data by averaging performance across different splits, capturing a wider range of potential data variations.


2. Less Variability:

A single train-test split can produce a high variance estimate due to random selection of data. Some splits may contain easier or more difficult examples, which skews the accuracy. Cross-validation reduces this variability by averaging the results across multiple splits, leading to more consistent and representative estimates of test accuracy.


3. Reduced Risk of Overfitting:

When training on a single split, the model might overfit to that particular train set. Cross-validation helps reduce the chance of overfitting by testing the model on multiple, different validation sets.

With more training and validation cycles, the averaged accuracy more closely reflects the model's true generalization ability, which typically aligns better with test performance.


4. Mitigating the Risk of Underfitting:

If a single split has an underrepresented pattern or class, it might not capture the complete distribution of the data, leading to underfitting and lower performance. By spreading the data across multiple splits in cross-validation, the model learns from a more representative dataset.


5. Test Accuracy Estimation in Different Scenarios:

When Data is Limited: In scenarios where the dataset is small, cross-validation maximizes the use of the data. Training on as much data as possible improves the accuracy estimation for the test set since the model has more exposure to patterns within the data.

With Large Datasets: Even for large datasets, cross-validation improves the stability of the accuracy estimate, especially if the data has inherent variability (e.g., MNIST digits can have slight variations depending on style, thickness, etc.).


6. Practical Outcomes:

Studies and practical experiments often show that cross-validation produces estimates that are closer to the actual test accuracy than single-split evaluations. Although the cross-validation accuracy might not perfectly match the test accuracy, it usually falls within a narrower range and is more robust.

Limitations:

Computational Cost: Cross-validation requires training the model multiple times, which can be expensive, especially for large datasets or complex models. However, this trade-off is usually worth it for a more accurate and reliable estimate.

Still Needs a Separate Test Set: It's important to note that cross-validation estimates validation accuracy, not test accuracy. You should still keep a separate test set to evaluate the final model and ensure it’s completely unseen by the model during both training and validation.


Conclusion:

In summary, cross-validation gives a more accurate estimate of test accuracy compared to using a single split. It mitigates the impact of random splits, improves generalization, and provides a better understanding of how the model might perform on unseen data.

Would you like to see an example implementation or further details about cross-validation techniques?

Yes, increasing the number of iterations in cross-validation (or any iterative validation process) generally leads to a more accurate and stable estimate of test accuracy. However, the relationship between the number of iterations and the quality of the estimate is not linear, and there are diminishing returns after a certain point. Let's break this down:

1. More Iterations Reduce Variability

With more iterations, the model is trained and validated on different random splits of the data multiple times. This reduces the variability in the accuracy estimate caused by any particular random split.

Higher iterations allow the model to experience a wider variety of data configurations, making the final accuracy estimate more robust to fluctuations caused by the randomness in the data.


2. More Iterations Lead to a More Stable Estimate

The more times you train and test the model, the closer the average accuracy estimate gets to the true generalization performance of the model.

This is based on the law of large numbers, where, as the number of trials increases, the average result becomes closer to the expected value.

In cross-validation, this would mean that with more iterations (or folds in k-fold cross-validation), the accuracy estimate stabilizes and converges toward the true test accuracy of the model.


3. Diminishing Returns

After a certain point, increasing the number of iterations yields diminishing returns. Once the accuracy estimate stabilizes, adding more iterations improves the estimate only slightly.

For instance, in k-fold cross-validation, increasing the value of k (e.g., from 5 to 10 folds) improves the estimate but at a slower rate. The estimate might not improve significantly from 10-fold to 20-fold.

The trade-off here is that while more iterations lead to better estimates, the computational cost increases significantly because the model must be trained multiple times.


4. Trade-off Between Accuracy and Computational Cost

While more iterations lead to better estimates, they also require more computational resources and time, especially for large datasets or complex models. The choice of the number of iterations should balance improved estimate accuracy and the available computational power.

In most cases, values like 5-fold or 10-fold cross-validation are widely used because they provide a good balance between estimate accuracy and computation time.


5. Effect on Bias and Variance

Increasing the number of iterations can reduce the variance of the accuracy estimate. This is because more iterations smooth out the randomness of each individual split.

However, increasing the number of iterations does not reduce bias. The bias is determined by the model itself and the quality of the data. If the model consistently performs poorly on every fold, more iterations won’t help; instead, the model needs to be improved (e.g., by tuning hyperparameters or improving feature selection).


6. Monte Carlo Cross-Validation

In Monte Carlo cross-validation (or repeated random subsampling validation), multiple random train-test splits are generated, and the model is trained and evaluated on these splits repeatedly.

This method can be repeated indefinitely, and increasing the number of iterations leads to more accurate estimates, but again, the improvement diminishes with more iterations.


7. Stratified Cross-Validation and Number of Iterations

If the dataset is imbalanced, using stratified cross-validation with more iterations can also improve the estimate. By ensuring that each split maintains the same class distribution, the model gets a better estimate of its performance across different class distributions.

More iterations here allow for more robust accuracy estimates across both the majority and minority classes.


When Do More Iterations Help?

Small Datasets: More iterations help a lot with small datasets because each split can vary significantly. More iterations reduce the chance that any particular split skews the estimate.

High Variability in Data: If the data is noisy or has high variance, more iterations ensure the model has seen a wide variety of data and gives a more reliable performance estimate.

Less Computational Cost Concerns: If computational cost is not a major concern, increasing the number of iterations can lead to a more reliable estimate of accuracy.


Summary:

More iterations generally provide a more accurate and stable estimate of test accuracy.

However, after a certain point, the benefit diminishes, and you need to balance between improving the estimate and the computational cost.

Typically, 5-fold or 10-fold cross-validation is sufficient for most practical applications, but you can increase iterations further if you're working with noisy data, small datasets, or if computational cost is not an issue.


Would you like me to demonstrate this effect with a practical example or a specific dataset?

Yes, increasing the number of iterations, such as in cross-validation or repeated subsampling, can help when working with small training or validation datasets. However, there are important nuances and limitations to consider.

How Increasing Iterations Helps with Small Datasets:

1. Maximizes Data Utilization:

With a small dataset, you risk having unrepresentative splits when you use a single train-test split. Cross-validation, particularly k-fold cross-validation, allows the model to train and validate on different parts of the dataset, thereby making the most of the limited data.

By increasing the number of iterations, each data point will be used multiple times for both training and validation, which helps the model generalize better compared to a single split where some data points may never be used in training or validation.



2. Reduces the Effect of Outliers:

In small datasets, individual outliers or rare examples can disproportionately influence the model when included in a training or validation set. By increasing the number of iterations, you smooth out the influence of such outliers because they won’t dominate the result in every iteration.



3. More Representative Validation:

A small validation set might not be representative of the overall distribution, leading to unstable or misleading accuracy estimates. By repeatedly validating the model on different portions of the data (using k-fold or Monte Carlo cross-validation), you reduce the chance that a particularly lucky or unlucky validation set skews the performance estimate.

For example, in Monte Carlo cross-validation, repeated random splits of the data ensure the model sees different train-test configurations, leading to a more robust measure of validation accuracy.



4. Better Generalization:

With a small training set, the model may not capture the complete distribution of the data. Repeating the process with multiple iterations allows the model to train on different combinations of the data, exposing it to more variability and reducing overfitting to a specific subset of the small dataset.




Limitations of Increasing Iterations with Small Datasets:

1. Bias Remains:

While more iterations help stabilize the variance of the model’s performance, it doesn't necessarily improve the bias of the model. If the small dataset does not contain enough relevant information, the model may still underperform regardless of how many iterations you run. The model can only learn as much as the data allows.

This means that increasing iterations doesn't compensate for a lack of data diversity or quality.



2. Computational Cost:

Cross-validation, particularly with a higher number of folds or iterations, can be computationally expensive, especially if the model is complex. For small datasets, this may not be a significant concern, but if you have a very large model (e.g., a deep neural network), repeating training many times may become impractical.



3. Overfitting Risk:

With very small datasets, the model may overfit to the training data, especially when validation sets are small as well. Cross-validation reduces overfitting by exposing the model to different splits, but if the training data itself is too small to capture the underlying patterns, the model may still overfit regardless of the number of iterations.



4. The Curse of Small Validation Sets:

Even with cross-validation, each fold will still have a small validation set if the overall dataset is small. While more iterations help reduce variability, there’s a limit to how much you can improve performance estimates when the validation set is small and potentially unrepresentative. This is particularly true for high-variance models.




Strategies for Very Small Datasets Beyond Increasing Iterations:

While increasing iterations can help stabilize performance estimates, there are other strategies you can use to better deal with very small datasets:

1. Data Augmentation:

For datasets like images (e.g., MNIST), you can apply data augmentation techniques to artificially increase the size of your training dataset by generating variations of the data (e.g., rotations, scaling, shifting).



2. Transfer Learning:

If the dataset is small, consider using a pre-trained model (e.g., from a large dataset like ImageNet) and fine-tuning it on your small dataset. This allows you to leverage learned features from larger datasets, improving the model’s performance on limited data.



3. Regularization:

Use regularization techniques such as L2 regularization, dropout, or early stopping to prevent overfitting when training on small datasets.



4. Bootstrapping:

Instead of cross-validation, you can use bootstrapping, where you sample the training data with replacement, allowing you to create multiple training sets. This can be especially helpful in situations where the

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.